In [1]:
import pandas as pd
import os
import csv
import json
""" Replace 14 with 15 in full file, select proper log file"""
data_dir = '../data/dktData/delft15/'
event_log = os.path.join('../data'+'/DelftX_AE1110x_2T2015-events.log')
# event_log = os.path.join('../data'+'/DelftX_AE1110x_1T2014-events.log')
import datetime
import pandas as pd
import json
import numpy as np
def process_time(time):
    return datetime.datetime.strptime(time[:-6], '%Y-%m-%dT%H:%M:%S.%f' if '.' in time[:-6] else '%Y-%m-%dT%H:%M:%S')

In [2]:

"""This will read log data and save problemcheck log events only"""
def read_raw_log_file(log_file=event_log):

    working_data = []
    fail_count = 0
    problem_check_count=0
    with open(log_file) as data_file:
        for line in data_file.readlines():
            try:
                data = json.loads(line)
                if data['event_type']=='problem_check':
                    working_data.append(data)

                    problem_check_count+=1
    #                     working_data.append(line)
            except:
                fail_count+=1
                print("Failed to read line:")
                print(line)
                continue
    print("Number of problem check log entries:", problem_check_count)
    print("Number of failed lines: ", fail_count)
    print("Length of successfuly read data: ", len(working_data))
    # SAVE LOG DATA  OF PROBLEM CHECK STUDENTS in a json file
    with open(data_dir+"delft_15_only_problem_check_log.json", 'w') as fout:
        json.dump(working_data, fout)
    return working_data
# working_data=read_raw_log_file()

In [3]:
# >>load only the problem check log data if the problem check log does nt exist 
#  run the above function: read_raw_log_file()
# import json
# data_problem_check = json.load(open(data_dir+'delft_15_only_problem_check_log.json'))

In [4]:
# for key in data_problem_check[0]:
#     print(key)
    
# # print("")

# for key in data_problem_check[0]["event"]:
#     print(key)

In [5]:
# SAVE STUDENT DATA TO DATAFRAME

# listOfRows=[]
# for row in data_problem_check:
#     rowdict={}
#     rowdict["time"]=process_time(row['time'])
#     rowdict["username"]=row['username']
# """for 2015 edit problemid using this"""
##    rowdict['problem_id']=row['event']['problem_id'].split("@")[-1])
#     rowdict["problem_id"]=row['event']['problem_id'].split("/")[-1]
#     rowdict["success"]=row['event']['success']
#     rowdict["attempts"]=row['event']['attempts']
#     listOfRows.append(rowdict)
  
    
# df = pd.DataFrame(listOfRows)
# df.to_csv(data_dir+'delft_15_DKTrequired_data.csv',index=False)


In [6]:
"""Read saved file of problem check events only"""

df=pd.read_csv(data_dir+'delft_15_DKTrequired_data.csv')
df.head()


,attempts,problem_id,success,time,username
0,1,61c79de13bf845a3afa9202995837c1a,correct,2015-06-02 19:17:18.359423,username_589385345
1,1,2053e2db1881424980ca3ce38a916437,correct,2015-06-02 19:18:37.946467,username_589385345
2,1,b8ca0effd8db42dba28d14a90cbc1163,correct,2015-06-02 19:20:14.095355,username_102109850
3,1,b8ca0effd8db42dba28d14a90cbc1163,incorrect,2015-06-02 19:21:55.483060,username_234891911
4,2,b8ca0effd8db42dba28d14a90cbc1163,correct,2015-06-02 19:22:05.874960,username_234891911


In [7]:
df.shape

(683319, 5)

In [8]:
unique_user_id =set(filter(lambda x: x is not None, df['username'].tolist()))

In [9]:
len(unique_user_id)

9242

In [10]:

# for username,group in df.groupby("username"):
# #     print(username,group)
#     group=group.sort_values('time')
# #     #convert sorted time to timestep
#     group["timestep"]=list(range(len(group["time"])))


In [11]:
# get idea about number of timesteps for each users
df.groupby('username')['time'].count().describe()

count    9242.000000
mean       73.936269
std       148.752446
min         1.000000
25%         5.000000
50%        15.000000
75%        41.000000
max       845.000000
Name: time, dtype: float64

In [12]:
"""CONVERT TIME TO TIMESTEPS"""
def create_timesteps_truncated(group,limit=100):
#     print(group)
    group=group.sort_values('time')
#     assign timesteps from 1..
    indices=list(range(1,len(group["time"])))
    length=len(indices)
    if length>limit:
#         print(length)
#         set all other timesteps to 0
        indices[limit:]=[0]*(length-limit)
    group["timesteps"]=indices
    return group


df=df.groupby("username").apply(lambda group : create_timesteps_truncated(group))
# df.to_csv("delft_15_problems_with_timestepnn.csv", index=False)

""" Also we can create more datapoints by 
breaking large sequences into small length 
i.e multiple sequences per student """

' Also we can create more datapoints by \nbreaking large sequences into small length \ni.e multiple sequences per student '

In [13]:
# for username,group in df.groupby("username"):
# # #     print(username,group)
# #     group=group.sort_values('time')
# #     #convert sorted time to timestep
#     group=create_timesteps(group)
#     print(group)
#     break

In [14]:
df.timesteps.describe()

count    683319.000000
mean         15.268159
std          26.072068
min           0.000000
25%           0.000000
50%           0.000000
75%          20.000000
max         100.000000
Name: timesteps, dtype: float64

In [15]:
# get rows where timstep is valid 
dfvalid=df[df['timesteps']!=0]
dfvalid.shape


(289402, 6)

In [16]:
dfvalid.head()

attempts                        problem_id  \
username                                                                
username_100663505 257108         1  e2ba3922c9374e089f30b068cd59eb85   
                   257109         1  a5e8909e66554554bd5517b8cf1fbb25   
                   255564         1  2d876aa861ae454589d96e80aa6e82fe   
                   255558         1  e45fa5e2a8e14f7cbdbe6e1a662a63c8   
                   255559         1  191a76b3412d4eb284583ab90ab16899   

                           success                        time  \
username                                                         
username_100663505 257108  correct  2015-09-17 20:15:19.860762   
                   257109  correct  2015-09-17 20:16:49.305596   
                   255564  correct  2015-09-17 20:22:07.486771   
                   255558  correct  2015-09-17 20:25:04.750014   
                   255559  correct  2015-09-17 20:26:25.969085   

                                     username  timesteps  
username                                                  
username_100663505 257108  username_100663505          1  
                   257109  username_100663505          2  
                   255564  username_100663505          3  
                   255558  username_100663505          4  
                   255559  username_100663505          5

In [17]:
assistment_df= dfvalid.pivot(index='username', columns='timesteps', values='problem_id')

In [18]:
assistment_df.shape

(9242, 100)

In [19]:
# assistment_df.isnull().sum()

### Replace id with integer numbers

In [11]:
# Convert problem Ids to unique number
def get_xml_files(path):
    """
    1st method Getting unique problem ids from xml of problem files
    """
    xml_list=[]
    xml_dict = {}
    file_no=0
    for filename in os.listdir(path):
# to ensure no two problem ids have same name
        if filename.endswith(".xml") and filename not in xml_list :
#save file name without 
            xml_list.append(filename[:-4]) 
            xml_dict[filename[:-4]]=file_no
            file_no+=1
            
    # save SKILL DICT FILE
    with open(data_dir+'skill_dict_delft.json', 'w') as fp:
        json.dump(problem_xml_dict, fp)
    return xml_dict


# problem_xml_dict_15=get_xml_files("../data/DelftX-AE1110x-2T2015/problem")
# problem_xml_dict=get_xml_files("../data/DelftX-AE1110x-1T2014/problem")
# correctness={"correct":int(1),"incorrect":int(0)}

# # # speed efficient than pandas unique command
# # """2nd method Getting unique problems from log data"""
# # unique_problem_id =set(filter(lambda x: x is not None, df['problem_id'].tolist()))
# # problem_id_dict={problem_id: int(index+1) for index, problem_id in enumerate(unique_problem_id)}

# #  USE SAME DICTIONARY FOR BOTH COURSES
# # with open(data_dir+'skill_dict_delft_15.json', 'w') as fp:
# #     json.dump(problem_id_dict, fp)
# # # set correctness dictionary
correctness={"correct":int(1),"incorrect":int(0)}


In [16]:
problem_id_dict

{u'0106323fe2f34b9dafd2dce56a9c6bf3': 15,
 u'0199bd01718b48b881d106c5690cf623': 5,
 u'019efe43d6f5434ba547bdb63aa4f218': 258,
 u'01a8694e166a497aa60e26f6aadfee0a': 120,
 u'02368d76692b4b0e8cff184ae08fea6e': 85,
 u'028b0ea764aa47eb9ef52507cba16459': 40,
 u'02de428bbe524545a85cc91d7a44dbc8': 16,
 u'03856c35c3904976bd829d2d3e18cbe7': 23,
 u'0473be6d3d1d4bec9f2778651743020e': 20,
 u'06f2d79163044288a6c8b2801b6b3c66': 308,
 u'090fdd1f76944fee8449bc07f4250bbf': 30,
 u'097c1f81e2a1482baba1198724095d16': 99,
 u'0b7a393864d842bd92fcd1082cc1186d': 288,
 u'0c0c6457408e4ab19f2fb2cea2f7f16b': 92,
 u'0caa402c9bbc495b8504c239d4c9de8a': 143,
 u'0e3d146751a24a93b5639f74ed3a072f': 309,
 u'0eb7f6fe2c7e452280fd75382286b77f': 220,
 u'0fa310e8660d497097c64c10818e4a6c': 108,
 u'0fb126958a83488698b4086eb7058afa': 167,
 u'106e84ede5c94f5f8c91d3d5e1c2a11c': 322,
 u'106f24b51a5440d58da99918a614e419': 290,
 u'10a9ca27349f4e698c5e259e92746090': 166,
 u'1119072395d7442d91369a8f7ed0dbf5': 277,
 u'13d37e77344046b8b89

In [17]:

with open('../code/data_dkt/'+'skill_dict_delft_all_0_336_all.json', 'w') as fp:
          json.dump(problem_id_dict, fp)



In [21]:
# s=0

# problem_id_dict=json.load(open('../data/dktData/delft14/skill_dict_delft_14.json'))
# for k in problem_xml_dict:
#     if k not in problem_xml_dict_15:
#         s+=1
#         print(k)
# print(s,'keys dont match')

# s=0
# for k in problem_xml_dict_15:
#     if k not in problem_xml_dict:
#         s+=1
#         print(k)
# print(s,'keys dont match')
# s=0
# for k in problem_id_dict:
#     if k not in problem_xml_dict_15:
#         s+=1
#         print(k)
# print(s,'keys dont match')


In [ ]:
# replace problem ids in dataframe above 
# error=0

# for index, row in dfvalid.iterrows():
#     try:
#         prob_id=row["problem_id"]
#         dfvalid.loc[index,'problem_id']=problem_id_dict[prob_id]
#         success=row['success']
#         dfvalid.loc[index,"success"]=correctness[success]
#     except:
#         error+=1
# print('Could not convert this number of problems: ',error)


       
        

# df.replace({"problem_id":problem_xml_dict})



In [1]:
problem_id_dict=json.load(open('../data/dktData/skill_dict_delft_all.json'))
skill_df=assistment_df.replace(problem_id_dict) 
skill_df=skill_df.fillna(0)
skill_df=skill_df.astype(int)

NameError: name 'json' is not defined

In [ ]:
skill_df.shape

In [ ]:
skill_df.head()

In [ ]:
# skill_df.isnull().sum()

In [ ]:

# skill_df.to_csv(data_dir+'skill_df_delft_15.csv')

In [23]:
response_data= dfvalid.pivot(index='username', columns='timesteps', values='success')

In [26]:
response_df=response_data.replace(correctness) 


In [ ]:
# response_df.head()

In [27]:
response_df=response_df.fillna(0)
response_df=response_df.astype(int)

In [28]:
# response_df.to_csv(data_dir+'response_df_delft_15.csv')

In [ ]:
# COMBINE DATA:
# SKILL14=pd.read_csv("../data/dktData/delft15/")